In [1]:
import mitsuba
mitsuba.set_variant('gpu_autodiff_rgb')

# The C++ type associated with 'Float' is enoki::DiffArray<enoki::CUDAArray<float>>
from mitsuba.core import Float
import enoki as ek

# Initialize a dynamic CUDA floating point array with some values
x = Float([1, 2, 3])

# Tell Enoki that we'll later be interested in gradients of
# an as-of-yet unspecified objective function with respect to 'x'
ek.set_requires_gradient(x)

# Example objective function: sum of squares
y = ek.hsum(x * x)

# Now back-propagate gradient wrt. 'y' to input variables (i.e. 'x')
ek.backward(y)

# Prints: [2, 4, 6]
print(ek.gradient(x))

2021-09-07 21:19:48 INFO main [optix_api.cpp:56] Dynamic loading of the Optix library ..

[2, 4, 6]


In [2]:
import enoki as ek
import mitsuba
mitsuba.set_variant('gpu_autodiff_rgb')

from mitsuba.core import Thread
from mitsuba.core.xml import load_file
from mitsuba.python.util import traverse
from mitsuba.python.autodiff import render, write_bitmap
from mitsuba.core import Color3f

# Load the Cornell Box
Thread.thread().file_resolver().append('C:/Users/Max/Code/mitsuba2/resources/data/scenes/cbox/')
scene = load_file('C:/Users/Max/Code/mitsuba2/resources/data/scenes/cbox/cbox.xml')

# Find differentiable scene parameters
params = traverse(scene)

param_ref = Color3f(params['red.reflectance.value'])
params.keep(['red.reflectance.value'])
print(param_ref)
print(params)

2021-09-07 21:19:48 INFO main [xml.cpp:1221] Loading XML file "C:\Users\Max\Code\mitsuba2\resources\data\scenes\cbox\cbox.xml" ..

2021-09-07 21:19:48 INFO main [xml.cpp:1222] Using variant "gpu_autodiff_rgb"

2021-09-07 21:19:48 INFO main [xml.cpp:355] "C:\Users\Max\Code\mitsuba2\resources\data\scenes\cbox\cbox.xml": in-memory version upgrade (v2.0.0 -> v2.2.1) ..

2021-09-07 21:19:48 INFO main [PluginManager] Loading plugin "plugins\srgb.dll" ..

2021-09-07 21:19:48 INFO main [PluginManager] Loading plugin "plugins\srgb_d65.dll" ..

...

2021-09-07 21:19:48 INFO main [PluginManager] Loading plugin "plugins\area.dll" ..

2021-09-07 21:19:48 INFO main [PluginManager] Loading plugin "plugins\uniform.dll" ..

2021-09-07 21:19:48 INFO main [PluginManager] Loading plugin "plugins\obj.dll" ..

2021-09-07 21:19:48 INFO main [Scene] Building scene in OptiX ..

[[0.569717, 0.0430139, 0.0443233]]
ParameterMap[
  * red.reflectance.value,
]


In [3]:
image_ref = render(scene, spp=8)
crop_size = scene.sensors()[0].film().crop_size()
write_bitmap('temp/out_ref.png', image_ref, crop_size)

![out_ref.png](out_ref.png)

In [5]:
params['red.reflectance.value'] = [.9, .9, .9]
params.update()

from mitsuba.python.autodiff import Adam
opt = Adam(params, lr=.2)

image = render(scene, spp=8)
crop_size = scene.sensors()[0].film().crop_size()
write_bitmap('temp/out_start.png', image, crop_size)

for it in range(100):
    # Perform a differentiable rendering of the scene
    image = render(scene, optimizer=opt, unbiased=True, spp=1)

    # Objective: MSE between 'image' and 'image_ref'
    ob_val = ek.hsum(ek.sqr(image - image_ref)) / len(image)

    # Back-propagate errors to input parameters
    ek.backward(ob_val)

    # Optimizer: take a gradient step
    opt.step()

    err_ref = ek.hsum(ek.sqr(param_ref - params['red.reflectance.value']))
    print('Iteration %03i: error=%g' % (it, err_ref[0]))

print(param_ref)
print(params['red.reflectance.value'])

image = render(scene, spp=8)
crop_size = scene.sensors()[0].film().crop_size()
write_bitmap('temp/out_end.png', image, crop_size)


Iteration 000: error=0.879108
Iteration 001: error=0.427435
Iteration 002: error=0.17983
Iteration 003: error=0.0704628
Iteration 004: error=0.051545
Iteration 005: error=0.0358767
Iteration 006: error=0.0175408
Iteration 007: error=0.00573612
Iteration 008: error=0.00469874
Iteration 009: error=0.0118884
Iteration 010: error=0.0199318
Iteration 011: error=0.0220037
Iteration 012: error=0.0176046
Iteration 013: error=0.0105169
Iteration 014: error=0.00519656
Iteration 015: error=0.00386257
Iteration 016: error=0.00586938
Iteration 017: error=0.00919437
Iteration 018: error=0.0116088
Iteration 019: error=0.0113934
Iteration 020: error=0.00905004
Iteration 021: error=0.00587572
Iteration 022: error=0.00335412
Iteration 023: error=0.00242183
Iteration 024: error=0.00306424
Iteration 025: error=0.00420559
Iteration 026: error=0.00444144
Iteration 027: error=0.00358264
Iteration 028: error=0.00204941
Iteration 029: error=0.00102251
Iteration 030: error=0.00116259
Iteration 031: error=0.0023

![start](temp/out_start.png)
![end](temp/out_end.png)